In [2]:
using Images
using PyPlot
using Test
using LinearAlgebra
using FileIO

# Transform from Cartesian to homogeneous coordinates
function cart2hom(points::Array{Float64,2})
    
    points_hom = vcat(points, ones(1, size(points, 2)))

  return points_hom::Array{Float64,2}
end

# Transform from homogeneous to Cartesian coordinates
function hom2cart(points::Array{Float64,2})
    
    points_cart = points[1:end-1, :] ./ points[end: end, :]
  return points_cart::Array{Float64,2}
end

# Translation by v
function gettranslation(v::Array{Float64,1})
    T = Matrix{Float64}(I, 4, 4)
    T[1:3, end] = v
  return T::Array{Float64,2}
end

# Rotation of d degrees around x axis
function getxrotation(d::Int)
    theta = d / 180 * pi
    Rx_cart = [1 0 0; 0 cos(theta) -sin(theta); 0 sin(theta) cos(theta)]
    Rx = vcat(hcat(Rx_cart, zeros(3, 1)), [0 0 0 1])
  return Rx::Array{Float64,2}
end

# Rotation of d degrees around y axis
function getyrotation(d::Int)
    theta = d / 180 * pi
    Ry_cart = [cos(theta) 0 -sin(theta); 0 1 0; sin(theta) 0 cos(theta)]
    Ry = vcat(hcat(Ry_cart, zeros(3, 1)), [0 0 0 1])
  return Ry::Array{Float64,2}
end

# Rotation of d degrees around z axis
function getzrotation(d::Int)
    theta = d / 180 * pi
    Rz_cart = [cos(theta) -sin(theta) 0; sin(theta) cos(theta) 0; 0 0 1]
    Rz = vcat(hcat(Rz_cart, zeros(3, 1)), [0 0 0 1])
  return Rz::Array{Float64,2}
end

# Central projection matrix (including camera intrinsics)
function getcentralprojection(principal::Array{Int,1}, focal::Int)
    K = [focal 0. principal[1] 0.; 0. focal principal[2] 0.; 0. 0. 1. 0.]
    
  return K::Array{Float64,2}
end


# Return full projection matrix P and full model transformation matrix M
function getfullprojection(T::Array{Float64,2},Rx::Array{Float64,2},Ry::Array{Float64,2},Rz::Array{Float64,2},V::Array{Float64,2})
    
    M = Rz * Rx * Ry * T
#    M = Rx * Ry * Rz * T
#    M = T * Rz * Rx * Ry
    P = V * M
  return P::Array{Float64,2},M::Array{Float64,2}
end

# Load 2D points
function loadpoints()
    points = load("obj2d.jld2", "x")
  return points::Array{Float64,2}
end

# Load z-coordinates
function loadz()
    z = load("zs.jld2", "Z")
  return z::Array{Float64,2}
end

# Invert just the central projection P of 2d points *P2d* with z-coordinates *z*
function invertprojection(P::Array{Float64,2}, P2d::Array{Float64,2}, z::Array{Float64,2})
    P3d = inv(P[:, 1:3]) * cart2hom(P2d).*z
  return P3d::Array{Float64,2}
end


# Invert just the model transformation of the 3D points *P3d*
function inverttransformation(A::Array{Float64,2}, P3d::Array{Float64,2})
    X = inv(A) * cart2hom(P3d)
  return X::Array{Float64,2}
end

# Plot 2D points
function displaypoints2d(points::Array{Float64,2})
    figure()
    plot(points[1, :], points[2, :])
    
  return gcf()::Figure
end

# Plot 3D points
function displaypoints3d(points::Array{Float64,2})
    figure()
    scatter3D(points[1,:], points[2, :], points[3, :])
    title("3D scene")
  return gcf()::Figure
end

# Apply full projection matrix *C* to 3D points *X*
function projectpoints(P::Array{Float64,2}, X::Array{Float64,2})
    
    P2d = hom2cart(P * cart2hom(X))
    
  return P2d::Array{Float64,2}
end



#= Problem 2
Projective Transformation =#

function problem3()
  # parameters
  t               = [6.7; -10; 4.2]
  principal_point = [9; -7]
  focal_length    = 8

  # model transformations
  T = gettranslation(t)
  Ry = getyrotation(-45)
  Rx = getxrotation(120)
  Rz = getzrotation(-10)

  # central projection including camera intrinsics
  K = getcentralprojection(principal_point,focal_length)

  # full projection and model matrix
  P,M = getfullprojection(T,Rx,Ry,Rz,K)

  # load data and plot it
  points = loadpoints()
  displaypoints2d(points)

  # reconstruct 3d scene
  z = loadz()
  Xt = invertprojection(K,points,z)
  Xh = inverttransformation(M,Xt)

  worldpoints = hom2cart(Xh)
  displaypoints3d(worldpoints)

  # reproject points
  points2 = projectpoints(P,worldpoints)
  displaypoints2d(points2)

  @test points ≈ points2
    #### If they are similar, return true
  return sum((points-points2).^2) / size(points, 2)      
end

problem3()

#=
Since 1/z is a scaling factor which differs at different points.


Order between rotation and translation does matter. Since with different model transformation matrix M, we
get different mean squared error 

M = Rz * Rx * Ry * T   ==>   1.2352846488547722e-29
M = Rx * Ry * Rz * T   ==>   4.005408824169259e-30
M = T * Rz * Rx * Ry   ==>   1.504826244588134e-29
=#


DimensionMismatch: DimensionMismatch("arrays could not be broadcast to a common size")